In [1]:
%load_ext autoreload

In [2]:
import numpy as np
from functools import partial
import visualization_utils as viz_utils
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from scipy.linalg import block_diag
import matplotlib.pyplot as plt
from pathlib import Path
import os

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
from pydrake.all import RigidTransform, RollPitchYaw, RevoluteJoint
from pydrake.all import RotationMatrix
import pydrake.symbolic as sym
from pydrake.solvers import MosekSolver, CommonSolverOption, SolverOptions, ScsSolver

import time

from pydrake.all import RationalForwardKinematics
from pydrake.geometry.optimization import HPolyhedron, Hyperellipsoid
from pydrake.geometry.optimization_dev import CspaceFreePath


In [4]:
import logging
drake_logger = logging.getLogger("drake")
drake_logger.setLevel(logging.DEBUG)

# Build and set up the visualization the plant and the visualization of the C-space obstacle

Note that running this cell multiple times will establish multiple meshcat instances which can fill up your memory. It is a good idea to call "pkill -f meshcat" from the command line before re-running this cell


In [5]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)




pendulum_world = FindResourceOrThrow("drake/C_Iris_Examples/assets/single_pendulum_world.urdf")

models = []
models.append(parser.AddModelFromFile(pendulum_world))

plant.Finalize()
Ratfk = RationalForwardKinematics(plant)

# the point about which we will take the stereographic projections
q_star = np.zeros(1)

t0 = time.time()
cspace_free_path = CspaceFreePath(plant, scene_graph, q_star, maximum_path_degree = 1, plane_order = 1)
t1 = time.time()
print(f"Time to construct line certifier = {t1-t0}s")

# #compute limits in t-space
limits_q = [plant.GetPositionLowerLimits().item(), plant.GetPositionUpperLimits().item()]
limits_s = [Ratfk.ComputeSValue(np.array([q]), q_star) for q in limits_q]
do_viz = True

# This line builds the visualization. Change the viz_role to Role.kIllustration if you
# want to see the plant with its illustrated geometry or to Role.kProximity
visualizer = IrisPlantVisualizer(plant, builder, scene_graph, cspace_free_path, viz_role=Role.kIllustration)


DEBUG:drake:FindResource ignoring DRAKE_RESOURCE_ROOT because it is not set.
DEBUG:drake:FindRunfile mechanism = RUNFILES_{MANIFEST_FILE,DIR}
DEBUG:drake:cwd = "/home/amice/Documents/coding_projects/drake/C_Iris_Examples"
DEBUG:drake:FindRunfile found by-manifest '/home/amice/Documents/coding_projects/drake/.drake-find_resource-sentinel' (good) and by-directory '/home/amice/.cache/bazel/_bazel_amice/32fdbbecfa8a7ce8feece95e48c42006/execroot/drake/bazel-out/k8-opt/bin/C_Iris_Examples/simple_pendulum_example.runfiles/drake/.drake-find_resource-sentinel' (good)
DEBUG:drake:FindRunfile found by-manifest '/home/amice/Documents/coding_projects/drake/package.xml' (good) and by-directory '/home/amice/.cache/bazel/_bazel_amice/32fdbbecfa8a7ce8feece95e48c42006/execroot/drake/bazel-out/k8-opt/bin/C_Iris_Examples/simple_pendulum_example.runfiles/drake/package.xml' (good)
DEBUG:drake:FindResource ignoring DRAKE_RESOURCE_ROOT because it is not set.
DEBUG:drake:FindRunfile found by-manifest '/home/am

Time to construct line certifier = 0.002148866653442383s
RATIONALS GENERATED
(-1 + 1.5 * a_12 + 0.5 * a_22 + 5 * a_32 + b_2)*1 + (1.5 * a_11 + 0.5 * a_21 + 5 * a_31 + b_1)*mu + (2 * ( - 5 * a_12 + 1.5 * a_32))*s[0] + (2 * ( - 5 * a_11 + 1.5 * a_31))*s[0] * mu + (-1 - 1.5 * a_12 + 0.5 * a_22 - 5 * a_32 + b_2)*s[0]^2 + ( - 1.5 * a_11 + 0.5 * a_21 - 5 * a_31 + b_1)*s[0]^2 * mu

(-1 + 1.5 * a_12 + 0.5 * a_22 + 5 * a_32 + b_2 - 10 * (s_0(mu_1)_coeff * a_12) + 3 * (s_0(mu_1)_coeff * a_32) - 1.5 * (pow(s_0(mu_1)_coeff, 2) * a_12) + 0.5 * (pow(s_0(mu_1)_coeff, 2) * a_22) - 5 * (pow(s_0(mu_1)_coeff, 2) * a_32) + (pow(s_0(mu_1)_coeff, 2) * b_2) - pow(s_0(mu_1)_coeff, 2))*1 + (1.5 * a_11 + 0.5 * a_21 + 5 * a_31 + b_1 - 2 * (s_0(mu_0)_coeff * s_0(mu_1)_coeff) - 3 * (s_0(mu_0)_coeff * s_0(mu_1)_coeff * a_12) + (s_0(mu_0)_coeff * s_0(mu_1)_coeff * a_22) - 10 * (s_0(mu_0)_coeff * s_0(mu_1)_coeff * a_32) + 2 * (s_0(mu_0)_coeff * s_0(mu_1)_coeff * b_2) - 10 * (s_0(mu_0)_coeff * a_12) + 3 * (s_0(mu_0)_c

## Set up the sliders so we can move the plant around manually

In [6]:
sliders = []
sliders.append(widgets.FloatSlider(min=plant.GetPositionLowerLimits().item(),
                                   max=plant.GetPositionUpperLimits().item(), 
                                   value=0, description='q0'))
# sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = np.array([0])
def handle_slider_change(change, idx):
    q[idx] = change['new']
    visualizer.show_res_q(q)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)


# visualizer.jupyter_cell()

Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


In [7]:
cert_options = CspaceFreePath.FindSeparationCertificateGivenPathOptions()
cert_options.terminate_segment_certification_at_failure = False

cert_options.num_threads = -1
cert_options.verbose = False
cert_options.solver_id = MosekSolver.id()
cert_options.solver_options = SolverOptions()
cert_options.terminate_path_certification_at_failure = False


In [8]:
geoms= cspace_free_path.plane_geometries_on_path()

In [9]:
geoms[0].positive_side_conditions[0].get_poly()

<Polynomial "(-1 + 1.5 * a_12 + 0.5 * a_22 + 5 * a_32 + b_2 - 10 * (s_0(mu_1)_coeff * a_12) + 3 * (s_0(mu_1)_coeff * a_32) - 1.5 * (pow(s_0(mu_1)_coeff, 2) * a_12) + 0.5 * (pow(s_0(mu_1)_coeff, 2) * a_22) - 5 * (pow(s_0(mu_1)_coeff, 2) * a_32) + (pow(s_0(mu_1)_coeff, 2) * b_2) - pow(s_0(mu_1)_coeff, 2))*1 + (1.5 * a_11 + 0.5 * a_21 + 5 * a_31 + b_1 - 2 * (s_0(mu_0)_coeff * s_0(mu_1)_coeff) - 3 * (s_0(mu_0)_coeff * s_0(mu_1)_coeff * a_12) + (s_0(mu_0)_coeff * s_0(mu_1)_coeff * a_22) - 10 * (s_0(mu_0)_coeff * s_0(mu_1)_coeff * a_32) + 2 * (s_0(mu_0)_coeff * s_0(mu_1)_coeff * b_2) - 10 * (s_0(mu_0)_coeff * a_12) + 3 * (s_0(mu_0)_coeff * a_32) - 10 * (s_0(mu_1)_coeff * a_11) + 3 * (s_0(mu_1)_coeff * a_31) - 1.5 * (pow(s_0(mu_1)_coeff, 2) * a_11) + 0.5 * (pow(s_0(mu_1)_coeff, 2) * a_21) - 5 * (pow(s_0(mu_1)_coeff, 2) * a_31) + (pow(s_0(mu_1)_coeff, 2) * b_1))*mu + ( - 3 * (s_0(mu_0)_coeff * s_0(mu_1)_coeff * a_11) + (s_0(mu_0)_coeff * s_0(mu_1)_coeff * a_21) - 10 * (s_0(mu_0)_coeff * s_0(mu

In [10]:
planes = cspace_free_path.separating_planes()
plane = planes[0]

In [11]:
link_geomtery = plane.negative_side_geometry
expressed_body = plane.expressed_body

In [12]:
pose = Ratfk.CalcBodyPoseAsMultilinearPolynomial(np.zeros(1), 
                                          link_geomtery.body_index(),
                                         expressed_body)

In [13]:
pose.rotation()

array([[<Polynomial "1*1">, <Polynomial "0">, <Polynomial "0">],
       [<Polynomial "0">, <Polynomial "1*1">, <Polynomial "0">],
       [<Polynomial "0">, <Polynomial "0">, <Polynomial "1*1">]],
      dtype=object)

DEBUG:drake:Meshcat connection opened from 0000:0000:0000:0000:0000:0000:0000:0001
DEBUG:drake:Meshcat connection closed from 0000:0000:0000:0000:0000:0000:0000:0001
DEBUG:drake:Meshcat connection opened from 0000:0000:0000:0000:0000:0000:0000:0001


In [14]:
cspace_free_path.path()

{Variable('s[0]', Continuous): <Polynomial "s_0(mu_1)_coeff*1 + s_0(mu_0)_coeff*mu">}